#Exploring Treebank Tagged Corpus

In [1]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
# reading the Treebank tagged sentences
nltk.download('treebank')
wsj = list(nltk.corpus.treebank.tagged_sents()) #import the wall street jounral corpus

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [3]:
# first few tagged sentences
print(wsj[:5])

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], [('Rudolph', 'NNP'), ('Agnew', 'NNP'), (',', ','), ('55', 'CD'), ('years', 'NNS'), ('old', 'JJ'), ('and', 'CC'), ('former', 'JJ'), ('chairman', 'NN'), ('of', 'IN'), ('Consolidated', 'NNP'), ('Gold', 'NNP'), ('Fields', 'NNP'), ('PLC', 'NNP'), (',', ','), ('was', 'VBD'), ('named', 'VBN'), ('*-1', '-NONE-'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('of', 'IN'), ('this', 'DT'), ('British', 'JJ'), ('industrial', 'JJ'), ('conglomerate', 'NN'), ('.', '.')], [('A', 'DT'), ('f

In [4]:
#setting the seed
random.seed(1234) 

# Splitting into train and test
train_set, test_set = train_test_split(wsj,test_size=0.3)

print(len(train_set))
print(len(test_set))
print(train_set[:5])

2739
1175
[[('``', '``'), ('The', 'DT'), ('sustained', 'VBN'), ('level', 'NN'), ('of', 'IN'), ('confidence', 'NN'), ('can', 'MD'), ('be', 'VB'), ('attributed', 'VBN'), ('*-3', '-NONE-'), ('to', 'TO'), ('the', 'DT'), ('continued', 'VBN'), ('favorable', 'JJ'), ('circumstances', 'NNS'), ('which', 'WDT'), ('*T*-1', '-NONE-'), ('affect', 'VBP'), ('the', 'DT'), ('consumer', 'NN'), ("'s", 'POS'), ('day-to-day', 'JJ'), ('economic', 'JJ'), ('life', 'NN'), (',', ','), ("''", "''"), ('said', 'VBD'), ('*T*-2', '-NONE-'), ('Mr.', 'NNP'), ('Linden', 'NNP'), ('.', '.')], [('The', 'DT'), ('final', 'JJ'), ('vote', 'NN'), ('came', 'VBD'), ('after', 'IN'), ('the', 'DT'), ('House', 'NNP'), ('rejected', 'VBD'), ('Republican', 'JJ'), ('efforts', 'NNS'), ('*', '-NONE-'), ('to', 'TO'), ('weaken', 'VB'), ('the', 'DT'), ('bill', 'NN'), ('and', 'CC'), ('approved', 'VBD'), ('two', 'CD'), ('amendments', 'NNS'), ('sought', 'VBN'), ('*', '-NONE-'), ('by', 'IN'), ('organized', 'VBN'), ('labor', 'NN'), ('.', '.')], [(

In [5]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

69900

In [6]:
train_tagged_words[:10]

[('``', '``'),
 ('The', 'DT'),
 ('sustained', 'VBN'),
 ('level', 'NN'),
 ('of', 'IN'),
 ('confidence', 'NN'),
 ('can', 'MD'),
 ('be', 'VB'),
 ('attributed', 'VBN'),
 ('*-3', '-NONE-')]

In [7]:
# visualize few tagged tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['``',
 'The',
 'sustained',
 'level',
 'of',
 'confidence',
 'can',
 'be',
 'attributed',
 '*-3']

In [8]:
# vocabulary
V = set(tokens)
print(len(V))

10181


In [9]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

46

In [10]:
print(T)

{'VB', '$', '-NONE-', 'RP', '-LRB-', 'PDT', 'POS', '``', 'VBZ', ',', 'MD', '#', 'DT', 'WP$', 'FW', 'WRB', 'NNS', 'UH', 'EX', 'PRP$', 'PRP', 'TO', '-RRB-', 'RB', ':', 'VBD', 'CD', 'JJ', 'VBP', 'SYM', 'RBR', 'JJS', '.', 'RBS', 'CC', 'WP', "''", 'VBN', 'NNP', 'VBG', 'JJR', 'LS', 'NN', 'NNPS', 'IN', 'WDT'}


#POS Tagging Algorithm - HMM

Hidden Markov Model based algorithm is used to tag the words. Given a sequence of words to be tagged, the task is to assign the most probable tag to the word.

In other words, to every word w, assign the tag t that maximises the likelihood P(t/w). Since P(t/w) = P(w/t). P(t) / P(w), after ignoring P(w), we have to compute P(w/t) and P(t).

P(w/t) is basically the probability that given a tag (say NN), what is the probability of it being w (say 'building'). This can be computed by computing the fraction of all NNs which are equal to w, i.e.

P(w/t) = count(w, t) / count(t).

The term P(t) is the probability of tag t, and in a tagging task, we assume that a tag will depend only on the previous tag. In other words, the probability of a tag being NN will depend only on the previous tag t(n-1). So for e.g. if t(n-1) is a JJ, then t(n) is likely to be an NN since adjectives often precede a noun (blue coat, tall building etc.).

Given the penn treebank tagged dataset, we can compute the two terms P(w/t) and P(t) and store them in two large matrices. The matrix of P(w/t) will be sparse, since each word will not be seen with most tags ever, and those terms will thus be zero.

#Emission Probabilities (i.e. probability of a word given a tag)

In [11]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [12]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

#Transition Probabilities (i.e. probability of getting a tag t2 given that the tag for previous word was t1)

In [13]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [14]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [15]:
tags_matrix

array([[1.6648169e-03, 9.4339624e-03, 7.7136517e-02, ..., 0.0000000e+00,
        1.0155383e-01, 1.1098780e-03],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0517091e-02, 2.4101664e-03, 7.4934267e-02, ..., 0.0000000e+00,
        1.4395267e-01, 2.1910605e-04],
       ...,
       [0.0000000e+00, 0.0000000e+00, 5.7803467e-03, ..., 5.7803467e-03,
        8.6705200e-02, 0.0000000e+00],
       [1.4753615e-04, 2.6851580e-02, 3.4966066e-02, ..., 1.7704337e-03,
        1.7556801e-02, 3.6884036e-03],
       [0.0000000e+00, 0.0000000e+00, 8.4666669e-01, ..., 0.0000000e+00,
        3.3333334e-03, 0.0000000e+00]], dtype=float32)

In [16]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [17]:
tags_df

,VB,$,-NONE-,RP,-LRB-,PDT,POS,``,VBZ,",",...,'',VBN,NNP,VBG,JJR,LS,NN,NNPS,IN,WDT
VB,0.001665,0.009434,0.077137,0.020533,0.001665,0.003330,0.000000,0.006659,0.000000,0.013319,...,0.001665,0.098224,0.032186,0.010544,0.006659,0.000000,0.064928,0.000000,0.101554,0.001110
$,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
-NONE-,0.010517,0.002410,0.074934,0.001315,0.001096,0.000000,0.000000,0.003067,0.037248,0.052366,...,0.000438,0.010736,0.037905,0.078659,0.001753,0.000000,0.018843,0.000000,0.143953,0.000219
RP,0.000000,0.006944,0.118056,0.000000,0.000000,0.000000,0.000000,0.020833,0.000000,0.027778,...,0.006944,0.000000,0.006944,0.000000,0.006944,0.000000,0.027778,0.000000,0.277778,0.000000
-LRB-,0.000000,0.164706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.047059,0.341176,0.000000,0.011765,0.000000,0.058824,0.000000,0.082353,0.000000
PDT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
POS,0.000000,0.007117,0.000000,0.000000,0.000000,0.000000,0.000000,0.007117,0.003559,0.005338,...,0.001779,0.012456,0.119217,0.007117,0.003559,0.000000,0.421708,0.007117,0.000000,0.000000
``,0.019646,0.000000,0.029470,0.000000,0.000000,0.000000,0.000000,0.001965,0.019646,0.000000,...,0.000000,0.017682,0.068762,0.005894,0.000000,0.000000,0.094303,0.000000,0.064833,0.000000
VBZ,0.002714,0.005427,0.194030,0.007463,0.000000,0.000678,0.000000,0.010176,0.000678,0.008141,...,0.001357,0.148575,0.020353,0.050882,0.006784,0.000000,0.040706,0.000000,0.084125,0.000000
",",0.001731,0.014137,0.034911,0.000000,0.000577,0.000000,0.000000,0.012406,0.026544,0.000000,...,0.057126,0.022793,0.142527,0.016734,0.002020,0.000289,0.050779,0.000577,0.077611,0.033179


In [18]:
# Visualizing one row of the tag probabilities 
tags_df.loc['.', :]

VB        0.000737
$         0.001474
-NONE-    0.021747
RP        0.000000
-LRB-     0.002949
PDT       0.001106
POS       0.000000
``        0.072982
VBZ       0.001843
,         0.000000
MD        0.000000
#         0.000000
DT        0.218209
WP$       0.000000
FW        0.000000
WRB       0.003686
NNS       0.042388
UH        0.000000
EX        0.003686
PRP$      0.005898
PRP       0.060450
TO        0.001474
-RRB-     0.004792
RB        0.041283
:         0.004055
VBD       0.000369
CD        0.008109
JJ        0.037228
VBP       0.000000
SYM       0.000000
RBR       0.000369
JJS       0.002212
.         0.000000
RBS       0.000000
CC        0.051235
WP        0.003686
''        0.058238
VBN       0.001474
NNP       0.174346
VBG       0.004792
JJR       0.003686
LS        0.002212
NN        0.046443
NNPS      0.001843
IN        0.114265
WDT       0.000369
Name: ., dtype: float32

In [19]:
!pip install hmmlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 4.1 MB/s eta 0:00:00


In [20]:
from hmmlearn import hmm
model = hmm.MultinomialHMM(n_components=len(T), algorithm='viterbi', random_state=42)


https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


In [ ]:
#Refernce:https://www.kaggle.com/code/trunganhdinh/hidden-markov-model-for-pos-tagging

#Viterbi Algorithm
Let's now use the computed probabilities P(w, tag) and P(t2, t1) to assign tags to each word in the document. We'll run through each word w and compute P(tag/w)=P(w/tag).P(tag) for each tag in the tag set, and then assign the tag having the max P(tag/w).

We'll store the assigned tags in a list of tuples, similar to the list 'train_tagged_words'. Each tuple will be a (token, assigned_tag). As we progress further in the list, each tag to be assigned will use the tag of the previous token.

Note: P(tag|start) = P(tag|'.')

In [22]:
len(train_tagged_words)

69900

In [23]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

#Evaluating on Test Set

In [24]:
# Running the Viterbi algorithm on a few sample sentences
# since running it on the entire data set will take many hours

random.seed(1234)
# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

[[('New', 'NNP'),
  ('York-based', 'JJ'),
  ('POP', 'NNP'),
  ('Radio', 'NNP'),
  ('provides', 'VBZ'),
  (',', ','),
  ('through', 'IN'),
  ('a', 'DT'),
  ('national', 'JJ'),
  (',', ','),
  ('in-store', 'JJ'),
  ('network', 'NN'),
  (',', ','),
  ('a', 'DT'),
  ('customized', 'VBN'),
  ('music', 'NN'),
  (',', ','),
  ('information', 'NN'),
  ('and', 'CC'),
  ('advertising', 'VBG'),
  ('service', 'NN'),
  ('which', 'WDT'),
  ('*T*-1', '-NONE-'),
  ('simulates', 'VBZ'),
  ('live', 'JJ'),
  ('radio', 'NN'),
  ('.', '.')],
 [('Until', 'IN'),
  ('now', 'RB'),
  (',', ','),
  ('however', 'RB'),
  (',', ','),
  ('buyers', 'NNS'),
  ('who', 'WP'),
  ('*T*-53', '-NONE-'),
  ('wanted', 'VBD'),
  ('*-1', '-NONE-'),
  ('to', 'TO'),
  ('finance', 'VB'),
  ('part', 'NN'),
  ('of', 'IN'),
  ('a', 'DT'),
  ('car', 'NN'),
  ('purchase', 'NN'),
  ('through', 'IN'),
  ('General', 'NNP'),
  ('Motors', 'NNPS'),
  ('Acceptance', 'NNP'),
  ('Corp.', 'NNP'),
  ('could', 'MD'),
  ("n't", 'RB'),
  ('put', 'VB

In [25]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [26]:
print("Time taken in seconds: ", difference)
print(tagged_seq)
#print(test_run_base)

Time taken in seconds:  99.39063787460327
[('New', 'NNP'), ('York-based', 'NNP'), ('POP', 'NNP'), ('Radio', 'NNP'), ('provides', 'VBZ'), (',', ','), ('through', 'IN'), ('a', 'DT'), ('national', 'JJ'), (',', ','), ('in-store', 'JJ'), ('network', 'NN'), (',', ','), ('a', 'DT'), ('customized', 'VB'), ('music', 'VB'), (',', ','), ('information', 'NN'), ('and', 'CC'), ('advertising', 'NN'), ('service', 'NN'), ('which', 'WDT'), ('*T*-1', '-NONE-'), ('simulates', 'VB'), ('live', 'VB'), ('radio', 'NN'), ('.', '.'), ('Until', 'IN'), ('now', 'RB'), (',', ','), ('however', 'RB'), (',', ','), ('buyers', 'NNS'), ('who', 'WP'), ('*T*-53', 'VB'), ('wanted', 'VBD'), ('*-1', '-NONE-'), ('to', 'TO'), ('finance', 'VB'), ('part', 'NN'), ('of', 'IN'), ('a', 'DT'), ('car', 'NN'), ('purchase', 'NN'), ('through', 'IN'), ('General', 'NNP'), ('Motors', 'NNPS'), ('Acceptance', 'VB'), ('Corp.', 'NNP'), ('could', 'MD'), ("n't", 'RB'), ('put', 'VB'), ('their', 'PRP$'), ('down', 'VB'), ('payment', 'NN'), ('on', 'IN'

In [27]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

In [28]:
accuracy = len(check)/len(tagged_seq)

In [29]:
accuracy

0.8692307692307693

https://www.kaggle.com/code/akshat0007/parts-of-speech-tagging-using-hmm